# Nihad's clustering neighborhoods Project

Importing libraries:

In [ ]:
import pandas as pd
import numpy as np
import requests

Setting the source Wikipedia's url value and calling the HTTP GET request on the url. Answer in the form of HTTP Response is saved into variable result. HTML code in textual representation can be found in attribute text:

In [8]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
result = requests.get(url)
result.text

Attribute text contains unformatted clear text which is unusable without knowledge of HTML. We know that our results of interest are located in the HTML object table, so webscraping is done by selecting the HTML code between opening and closing tag of the table. At the end of the cell is used method read_html() of pandas to get the content of the table in the pandas dataframe: 

In [96]:
idx1 = result.text.find('<table class="wikitable sortable"')
idx2 = result.text.find('</table>',idx1)
str = result.text[idx1:idx2+8]
dl = pd.read_html(str)
df_raw = dl[0]

Because original column name of the third column on the Wikipedia page is different than the name from the task, it will be renamed from 'Neighbourhood' to 'Neighborhood' 

In [ ]:
df_raw.columns.values[2]='Neighborhood'

Filtering of boroughs with 'Not assigned' value. Before and after the filtering operation the number of cells with those values of the column 'Borough' will be printed out: 

In [92]:
print('Control of not assigned boroughs before operation:',np.sum(df_raw['Borough']=='Not assigned'))
df_assigned = df_raw[df_raw['Borough']!='Not assigned']
df_assigned.reset_index(drop=True,inplace=True)
print('Control of not assigned boroughs after operation:',np.sum(df_assigned['Borough']=='Not assigned'))

Control of not assigned boroughs before operation: 77
Control of not assigned boroughs after operation: 0


Processing of neighborhoods with 'Not assigned' value. Before and after the assignment operation the number of cells with those values of the column 'Neighborhood' will be printed out: 

In [93]:
print('Control of not assigned neighborhoods before operation:',np.sum(df_assigned['Neighborhood']=='Not assigned'))
for i in range(0,df_assigned.shape[0]):
    if(df_assigned.iloc[i]['Neighborhood']=='Not assigned'):
       df_assigned.iloc[i]['Neighborhood'] = df_assigned.iloc[i]['Borough']
print('Control of not assigned neighborhoods after operation:',np.sum(df_assigned['Neighborhood']=='Not assigned'))

Control of not assigned neighborhoods before operation: 1
Control of not assigned neighborhoods after operation: 0


For the insight purposes we display the final dataframe:

In [94]:
df_assigned

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


At the end, the size of dataframe for the sake of control:

In [95]:
df_assigned.shape

(211, 3)